---
title: "Writing Stable Diffusion from Scratch 6"
reading-time: 
date: "2023-3-25"
categories: [fastaipart2,Stable-Diffusion]
---

What you should learn and practice after this : <br>
1- log , e rules 
2- what is softmax
3-
4- Train a simple linear model from scratch

So we're now up to a point where we can train a model. So let's do that mini batch training Notebook number four. So same first cell as before. We won't go through it. The cells also the same as before, so we won't go through it is the same model that we had before, so we won't go through it. So I just rerunning all that to see. Okay, 

so the first thing we should do, I think, is to improve our loss function. So it's not total rubbish anymore. So if you watch part one, you might recall that there are some Excel notebooks. One of those Excel notebooks is entropy example. Okay, so this is what we looked at. So just to remind you, what we're doing now is which we're saying, okay, rather than rather than outputting a single number for each image, we're going to instead output ten numbers for each image. And so that's going to be a one hot encoded set of it'll be like 1000, etc.. And so then that's going to be well, actually the outputs wont be 1000, they'll be basically probabilities won't they. So it'll be like .99 comma, you know, 3.01 etc. and the targets will be one hot encoded. So if it's the digit zero, for example, it might be 10000 that dot dot for all the ten possibilities. And so to see, you know, how good is it. So in this case, it's very good. It had 0.99 probability prediction that it's zero. And indeed it is, because this is this is the 100 coded version. And so the way we implement that is we don't even need to actually do the one hot encoding, thanks to some tricks, we can actually just directly store the integer, but we can treat it as if it's one hot encoded so we can just store the actual target zero as an integer. So the way we do that is we say, for example, for a single output, oh, it could be cat, let's say cat, dog, plain fish building the neural net spits out a bunch of outputs. What we do for softmax is we go e to of the power of each of those outputs. We sum up all of those e of the power offs. So here's the e of the power of each of those outputs is the sum of them. And then we divide one each one by the sum. So we divide each one by the sum that gives us a softMax's. And then for the lost function, we then compare the softMax's to the one encoded version. So let's say it was a dog that's going to have a one for dog and zero everywhere else. And then softMax, this is from this nice blog post here. This is the calculation, some of the ones and zeros. So each of the ones and zeros multiplied by the log of the probabilities. So here is the log probability times the actuals. And since the actual is either zero or one and only one of them is going to be a one, we're only going to end up with one value here. And so if we add them up, it's all zero except for one of them. So that's a cross entropy. So in the special case where the outputs one had encoded, then during the one hot encoded multiplied by the log, softmax is actually identical to simply saying, Oh, dog is in this row, let's just look it up directly and take its log soft max.

We can just index directly into it. So it's exactly the same thing. So that's just review. So if you haven't seen that before, then yeah, go and watch the part one video where we went into that in a lot more detail. Okay, So here's our softmax calculation. It's e to the power of each output divided by the sum of them all, we can use sigma notation to say exactly the same thing. And as you can see, Jupyter notebook lets us use latex. If you haven't used latex before, it's actually surprisingly easy to learn. You just put dollar signs around your equations like this and your equations backslash is going to be kind of like your functions if you like, and curly parentheses actually, Curly are used to kind of fit for arguments. So you can see here, here is e to the power of and then underscore as used just subscript. So this is x subscript i and power of is used for super scripts. So his dots you can see here it is dots so it's actually yeah learning like tech is easier than you might expect. It can be quite convenient for writing these functions when you want to. So anyway, that's what soft Max is as we'll see in a moment. Well, actually, as you've already seen in cross entropy, we don't really want soft Maxs. We want log of soft Macs. So a log of softMaxs is here it is. So we've got x.exp  divided by x.exp.sum and we're going to sum up over the last dimension and then we actually want to keep that dimension so that when we do the divided by, we want a trailing unit axis for exactly the same reason we saw when we did our MSE last function. So if you sum with keepdim = true, that leaves the unit axis in in that last position. So we don't have to put it back to avoid that horrible out of product issue. So this is the equivalent of this and then .log. So that's log of soft Maxs. So there is the log of the soft Maxs that are predictions. Now in terms of high school math, you may have forgotten, but you definitely are going to want to know a Key piece that in that list of things is is log and exponent rules. 

So check out Khan Academy or similar if you've forgotten them. But a quick reminder is, for example, the one we mentioned here log of A over B equals log of A minus log of B and equivalently log of A times B equals log of A plus log of B. And these are very handy because, for example, division can take a long time, multiply, can create really big numbers and have lots of floating point error. Being able to replace these things with pluses and minuses is very handy indeed. In fact, I used to give people an interview question which I did a lot of stuff with school and math school actually only has a some function for group by clauses. And I used to ask people how you would deal with calculate a compound interest column where the answer is basically that you have to say because compound interest is taking products. So it has be the sum of the log of the column and then e of the power of all that. So it's like all kinds of little places that these things coming in handy, but they come in to neural nets all the time. So we're going to take advantage of that because we've got a divided by it's being logged and also rather handily. We're going to have therefore the log of x.exp.log minus the log of this, but expand log opposites. So that is going to end up being x minus. So log soft max is just x minus, all this logged. And here it is, all this logged. So that's nice. So here's our simplified version. Okay, Now there's another very cool trick, which is one of these things I figured out myself and then discovered other people have known it for years. So not my trick, but it's always to rediscover things. The trick is what's written here. Let me explain what's going on. This piece here, the log of this sum right this sum here we've got x.exp.sum. Now x could be some pretty big numbers. And e of the power of that's going to be really big numbers. And e of the power of things creating really big numbers. Well, really big numbers. There's much less precision in your computers floating point handling the further, you get away from zero, basically. So we don't want really big numbers, particularly because we're going to be taking derivatives. And so if you're in an area that's not very precise as far as 13 point math is concerned, then the derivatives are going to be a disaster. They might even be zero because you've got two numbers that the computer can't even recognize is different.

So this is bad, but there's a nice trick we can do to make it a lot better. What we can do is we can calculate the max of the max of X, right. And we'll call that A. And so then rather than doing the log of the sum of e of x i, we're instead going to define A as being the maximum of all of our X values. It's our biggest number. Now, if we then subtract that from every number, that means none of the numbers are going to be big by definition because we have subtracted it from all of them. Now the problem is that's given us a different result. Right? But if you think about it, let's expand this sum. It's e to the power of x one , say plus e of the power of X to plus e of the power of x three and so forth. Okay, Now we just subtracted a from our exponents, which is we're now wrong. But I've got good news, I've got good news and bad is. The bad news is that you've got more high school math to remember, which is exponent rules. So X to the a plus b equals x to the a, times x to the b, and x to the b minus b equals x to the a divided by x to the b, And to convince yourself that's true, consider for example, two to the power of two three. What is that what you've got to do?

The power of two is just two times two and to the power of two plus three. Well, it's two times. Two times it is to the power of five. So you've got two to the power of two, you've got two of them here and you've got another three of them here. So we're just adding up the number to get the total index so we can take advantage of this here and say like, Oh well this is equal to a to the  e to the x1 over e to the a plus e to the x2 over e to the a ... And this is a common denominator. So we can put all that together  why do we do all that. Because if we now multiply that all by e of the a these would cancel out and we get the thing we originally wanted. So that means we simply have to multiply this by that. And this gives us exactly the same thing as we had before. 

But with critically, this no longer ever going to be a giant number, so this might seem a bit weird. We're doing extra calculations. It's not a simplification, it's a complexification, but it's one that's going to make it easier for our floating point unit. So that's our trick. It's rather than doing log of this sum, what we actually do is log of e of  times the sum of e to the X minus a, And since we've got a log of a product that's just a log, that's just the sum of the logs and log of e to the a so it's a plus that so this here is called

the log sum x trick people are pointing out. Thank you for that of course should have been inside the log you got this go sticking it on the outside like a crazy person. Yeah. Yeah, That's what I meant to say. Let's check if you've got any questions. Any questions yet? Okay, so here is the log.call it a (m in notebook) But anyway, so we find the maximum on the last dimension. And then here is the m plus that exact thing. Okay, so that's just another way of doing that. Okay, So that's the log, sum exp. So now we can rewrite log soft max as x minus log sum exp, and we're not going to use our evasion because PyTorch already has one, so we're just use pytorches.



In [ ]:
#| default_exp training

In [ ]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor,nn
import torch.nn.functional as F

In [ ]:
from fastcore.test import test_close

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [ ]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [ ]:
!ls -l data

total 16656
-rw-r--r-- 1 root root 17051982 Mar 27 05:00 mnist.pkl.gz


In [ ]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Initial setup

### Data

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x

In [ ]:
model = Model(m, nh, 10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

### Cross entropy loss

First, we will need to compute the softmax of our activations. This is defined by:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

or more concisely:

$$\hbox{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum\limits_{0 \leq j \lt n} e^{x_{j}}}$$ 

In practice, we will need the log of the softmax when we calculate the loss.

In [ ]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [ ]:
log_softmax(pred)

tensor([[-2.37, -2.49, -2.36,  ..., -2.31, -2.28, -2.22],
        [-2.37, -2.44, -2.44,  ..., -2.27, -2.26, -2.16],
        [-2.48, -2.33, -2.28,  ..., -2.30, -2.30, -2.27],
        ...,
        [-2.33, -2.52, -2.34,  ..., -2.31, -2.21, -2.16],
        [-2.38, -2.38, -2.33,  ..., -2.29, -2.26, -2.17],
        [-2.33, -2.55, -2.36,  ..., -2.29, -2.27, -2.16]], grad_fn=<LogBackward0>)

Note that the formula 

$$\log \left ( \frac{a}{b} \right ) = \log(a) - \log(b)$$ 

gives a simplification when we compute the log softmax:

In [ ]:
def log_softmax(x): return x - x.exp().sum(-1,keepdim=True).log()

Then, there is a way to compute the log of the sum of exponentials in a more stable way, called the [LogSumExp trick](https://en.wikipedia.org/wiki/LogSumExp). The idea is to use the following formula:

$$\log \left ( \sum_{j=1}^{n} e^{x_{j}} \right ) = \log \left ( e^{a} \sum_{j=1}^{n} e^{x_{j}-a} \right ) = a + \log \left ( \sum_{j=1}^{n} e^{x_{j}-a} \right )$$

where a is the maximum of the $x_{j}$.

In [ ]:
def logsumexp(x):
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

This way, we will avoid an overflow when taking the exponential of a big activation. In PyTorch, this is already implemented for us. 

In [ ]:
def log_softmax(x): return x - x.logsumexp(-1,keepdim=True)

In [ ]:
test_close(logsumexp(pred), pred.logsumexp(-1))
sm_pred = log_softmax(pred)
sm_pred

tensor([[-2.37, -2.49, -2.36,  ..., -2.31, -2.28, -2.22],
        [-2.37, -2.44, -2.44,  ..., -2.27, -2.26, -2.16],
        [-2.48, -2.33, -2.28,  ..., -2.30, -2.30, -2.27],
        ...,
        [-2.33, -2.52, -2.34,  ..., -2.31, -2.21, -2.16],
        [-2.38, -2.38, -2.33,  ..., -2.29, -2.26, -2.17],
        [-2.33, -2.55, -2.36,  ..., -2.29, -2.27, -2.16]], grad_fn=<SubBackward0>)

The cross entropy loss for some target $x$ and some prediction $p(x)$ is given by:

$$ -\sum x\, \log p(x) $$

But since our $x$s are 1-hot encoded (actually, they're just the integer indices), this can be rewritten as $-\log(p_{i})$ where i is the index of the desired target.

This can be done using numpy-style [integer array indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html#integer-array-indexing). Note that PyTorch supports all the tricks in the advanced indexing methods discussed in that link.

In [ ]:
y_train[:3]

tensor([5, 0, 4])

In [ ]:
sm_pred[0,5],sm_pred[1,0],sm_pred[2,4]

(tensor(-2.20, grad_fn=<SelectBackward0>),
 tensor(-2.37, grad_fn=<SelectBackward0>),
 tensor(-2.36, grad_fn=<SelectBackward0>))

In [ ]:
sm_pred[[0,1,2], y_train[:3]]

tensor([-2.20, -2.37, -2.36], grad_fn=<IndexBackward0>)

In [ ]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [ ]:
loss = nll(sm_pred, y_train)
loss

tensor(2.30, grad_fn=<NegBackward0>)

And if we check, we yeah, we go, it's our results. And so then as we've discussed, the cross entropy loss is the sum of the outputs times the log probabalities. And as we discussed, our outputs are one hot and coded or actually they're just the integers better still. So what we can do is we can I guess I should make that more clear. Actually the just the integer indices so we can simply rewrite that as negative log of the target. So that's what we had in our Excel. And so how do we do that in PyTorch? So this is quite interesting. There's A lot of cool things you can do with array indexing in PyTorch in Numpy. So basically they use the same approaches. Let's take a look. Here is the first three actual values in y_train. They're five zero and four. Now what we want to do is we want to find in our soft max predictions, we want to get five the fifth prediction in the zeroth row, the zeroth prediction in the first row and the fourth in the index two row. So these are the numbers that we want. This is going to be what we add up for the first few rows of last function. So how do we do that all in one go? Well, here's a cool trick. See here, I've got 0,1,2. If we index using a two lists, we can put here 0, 1,2, and for the second list we can put y_train[:3].   So it's going to be it's going to be zero comma 5, 1 comma zero, 2 comma four, which is, as you say, exactly the same thing. So therefore, this is actually giving us what we need for the cross entropy loss. So if we take range of our targets first dimension or zero index dimension, which is all this is and the target and then take the negative of that dot mean, that gives us our cross entropy loss, which is pretty neat in my opinion. All right. So pytorch calls this negative log likelihood loss and that's all it is. And so if we take the negative log likelihood and so get negative likelihood and we pass that to , the log soft max, then we get the loss. And this particular combination in PyTorch called F.across_entropy. So let's check. Yep. F.across_entropy gives us exactly the same thing. So that's cool. So we have now re-implemented cross entropy loss and there's a lot of confusing things going on there. A lot. And so this is one of those where you should pause the video and go back and look at each step and think not just like, what is it doing, but why is it doing it? And also try in lots of different values yourself to see if you can see what's going on and then put this aside and test yourself by reimplementing log soft max and nll_loss and cross entropy yourself and compare them to pytouches values. And so that's the piece of homework for you for this week

Then use PyTorch's implementation.

In [ ]:
test_close(F.nll_loss(F.log_softmax(pred, -1), y_train), loss, 1e-3)

In PyTorch, `F.log_softmax` and `F.nll_loss` are combined in one optimized function, `F.cross_entropy`.

In [ ]:
test_close(F.cross_entropy(pred, y_train), loss, 1e-3)

## Basic training loop

So now that we've got that we can actually create a training loop. So let's set our last function to be cross entropy. Let's create a batch size of 64. And so here's our first mini batch. Okay, so xb is the x mini batch. It's going to be from zero up to 64 from our training set, so we can now calculate our predictions. So that's 64 by ten. So for each of the 64 images in the mini batch, we have ten probability one for each digit. And our y(s) is just a let's print those out. So there's our first 64 target values. So these are the actual digits. And so our loss function. So we're going to start with a bad loss because it's entirely random at this point. Okay. So for each of the predictions 

we made, so those are our predictions. And so remember, those predictions are a 64 by ten. What did we predict? So for each one of these 64 rows, we have to go in and see where is the highest number. So if we go through here, we can go through each one. Here's a  0.123 So the highest number is this one. So you got to find the index of the highest number. The function to find the index of the highest number is called argmax. And here it is three. And I guess we could have also written this probably as preds.argmax. Normally you can do them either way. I actually prefer normally to do it this way. Yeah. There's the same thing. Okay. And the reason we want this is because we want to be at a complete accuracy. We don't need it for the actual neural net, but we just like to be able to see how we're going because it's like it's a metric. It's something that we use for understanding. So we take the argmax, we compare it to the actual. So that's going to give us a bunch of bools. If you turn those into floats, they'll be ones and zeros. The mean of those flights is the accuracy. So our current accuracy, not surprisingly, is around 10%. It's 9% because it's random. That's what you would expect. Let's train our first neural net. So we'll set a learning rate, which are a few epochs. So we're going to go through each epoch and we're going to go through from zero up to n. That's the 50,000 training rows and skipping by 64 the batch size each time. And so we're going to create a slice that starts at i So starting at zero and goes up 64 unless we're just got to n And so then we will slice into our training set for the x and for the y to get an x and y batches. We will then calculate our predictions, our loss function, and do it backward.

So the way I did this originally was I had all of these in  in separate cells and I just typed in, you know, i equals zero and then went through one cell at a time calculating each one until they all worked. And so then I can put them in a loop. Okay. So once we've got done backward, we can then with torch.no_grad go through each layer and if that's a layer that has weights, will update them to the existing weights minus the gradients times of learning and then zero out. So the weights and biases for the gradients, the gradients of the weights and biases this underscore means do it in place. So that sets this to zero. So if I run that up, it's going to run all of them. I guess I skipped. So there we go. It's finished. So you can see that our accuracy on the training sets have been unfair. But 23 epochs is 97%. So we now have a digit recognizer trains pretty quickly and it's not terrible at all. So that's a pretty good starting point. All right. So what we're going to do next time is we're going to refactor this training loop to make it dramatically dramatic training, dramatically simple step by step, until eventually we will get it down to where is it. So we'll get it down to something much, much shorter. And then we're going to add a validation set to it and a multi processing data loader. And then yeah, we'll be in a pretty good position, I think, to to start training some more interesting models. All right. Hopefully you found that useful and led some interesting things. And so what I'd really like you to do is at this point, now that you've kind of like got all these key basic pieces in place, is to really try to recreate them without picking as much as possible. So, you know, recreate your matrix, multiply, recreate those forward and backward passes, recreate something that steps through layers and even see if you can like recreate the idea of the dot forward and the dot backward. Make sure it's all in your head really clearly so that you fully understand what's going on. You know, at the very least, if you don't have time for that, because that's a big job, you could pick out a smaller part of that, the piece that you're more interested in, or you could just go through and look really at these notebooks. So if you go to kernel restart and clear output, it'll delete all the outputs and try to think like what are the shapes of things? Can you guess what they are, can you check them? And so forth. Okay, Thanks everybody. I hope you have a great week and I will see you next time by. 


Basically the training loop repeats over the following steps:
- get the output of the model on a batch of inputs
- compare the output to the labels we have and compute a loss
- calculate the gradients of the loss with respect to every parameter of the model
- update said parameters with those gradients to make them a little bit better

In [ ]:
loss_func = F.cross_entropy

In [ ]:
bs=50                  # batch size

xb = x_train[0:bs]     # a mini-batch from x
preds = model(xb)      # predictions
preds[0], preds.shape

(tensor([-0.09, -0.21, -0.08,  0.10, -0.04,  0.08, -0.04, -0.03,  0.01,  0.06], grad_fn=<SelectBackward0>),
 torch.Size([50, 10]))

In [ ]:
yb = y_train[0:bs]
yb

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7,
        6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3])

In [ ]:
loss_func(preds, yb)

tensor(2.30, grad_fn=<NllLossBackward0>)

In [ ]:
preds.argmax(dim=1)

tensor([3, 9, 3, 8, 5, 9, 3, 9, 3, 9, 5, 3, 9, 9, 3, 9, 9, 5, 8, 7, 9, 5, 3, 8, 9, 5, 9, 5, 5, 9, 3, 5, 9, 7, 5, 7, 9, 9, 3,
        9, 3, 5, 3, 8, 3, 5, 9, 5, 9, 5])

In [ ]:
#|export
def accuracy(out, yb): return (out.argmax(dim=1)==yb).float().mean()

In [ ]:
accuracy(preds, yb)

tensor(0.08)

In [ ]:
lr = 0.5   # learning rate
epochs = 3 # how many epochs to train for

In [ ]:
#|export
def report(loss, preds, yb): print(f'{loss:.2f}, {accuracy(preds, yb):.2f}')

In [ ]:
xb,yb = x_train[:bs],y_train[:bs]
preds = model(xb)
report(loss_func(preds, yb), preds, yb)

2.30, 0.08


In [ ]:
for epoch in range(epochs):
    for i in range(0, n, bs):
        s = slice(i, min(n,i+bs))
        xb,yb = x_train[s],y_train[s]
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias   -= l.bias.grad   * lr
                    l.weight.grad.zero_()
                    l.bias  .grad.zero_()
    report(loss, preds, yb)

0.12, 0.98
0.12, 0.94
0.08, 0.96
